In [ ]:
%%javascript
$('<div id="toc"></div>').css({position: 'fixed', top: '120px', left: 0}).appendTo(document.body);
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js');

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np

from jupyterthemes import jtplot
jtplot.style()

%matplotlib inline
%load_ext autotime
%load_ext line_profiler
%matplotlib inline 

# Data Load

In [ ]:
# Load data
properties_data = pd.read_csv('../Data/properties_2016.csv', low_memory=False)
# Load train data
train_data = pd.read_csv('../Data/train_2016_v2.csv')
# Elements to be forecasted - this is the framework
submission_sample = pd.read_csv('../Data/sample_submission.csv')
# Load label description and feature documentation
label_documentation = pd.read_csv('../Data/zillow_data_dictionary.csv', encoding='ISO8859_1')
# Replace null values, identify duplicates.
transactions = pd.merge(train_data, properties_data, how='left', on=['parcelid'])
duplicate_records = train_data[train_data['parcelid'].duplicated()]['parcelid'].unique()

In [ ]:
# Show how many nulls there are for each feature
properties_data.isnull().sum().sort_values(ascending=False).to_frame().plot.barh(figsize=(7,13), title='Nulls per feature')

# Visualisation

In [ ]:
transactions['latitude'] = transactions['latitude']/1000000
transactions['longitude'] = transactions['longitude']/1000000

# Get latitude and longitude extremes
min_lat = transactions['latitude'].min()
max_lat = transactions['latitude'].max()
min_lon = transactions['longitude'].min()
max_lon = transactions['longitude'].max()

# Build map
area = 0.1
fig = plt.figure(figsize=(40,40))
map = Basemap(projection='merc', lat_0 = np.mean([min_lat, max_lat]), lon_0 = np.mean([min_lon, max_lon]),
    resolution = 'h', area_thresh = 0.1,
    llcrnrlon=min_lon - area, llcrnrlat=min_lat - area,
    urcrnrlon=max_lon + area, urcrnrlat=max_lat + area)
 
map.drawcoastlines()
map.drawcountries()
map.fillcontinents(color = 'coral')
map.drawmapboundary()
 
lon = transactions['longitude'].values
lat = transactions['latitude'].values
x,y = map(lon, lat)
map.plot(x, y, 'bo', markersize=5)

In [ ]:
label_documentation

In [ ]:
fig = plt.figure(figsize=(200,50))
plt.plot(transactions['logerror'], linewidth=0.5)

# Processing

In [ ]:
def plot_data(test, pred, sample, title, width=40, height=10, linewidth=0.5, color1='white', color2='orange'):
    """ Plotting method. """
    fig = plt.figure(figsize=(width, height))
    plt.plot(pred[:sample], color=color1, zorder=4, linewidth=linewidth, label='%s Prediction'%(title))
    plt.plot(test[:sample], color=color2, zorder=3, linewidth=linewidth, label='%s True Data'%(title))
    plt.title = title
    plt.legend()

In [ ]:
# Some set of features which would intuitively make sense to be correlated with the price of the house.
# However, given that the problem is to discover areas of improvement in the model, they might not contribute
# significantly. 
temporal = ['month_of_year', 'quarter']#,'day_of_week','is_weekend']
ids      = ['storytypeid','airconditioningtypeid','buildingclasstypeid','typeconstructiontypeid',
            'architecturalstyletypeid','propertylandusetypeid','decktypeid','pooltypeid10',
            'pooltypeid2','buildingqualitytypeid','pooltypeid7','heatingorsystemtypeid',
            'code_fips','code_tract','code_block']
regional = ['regionidcounty','regionidcity','fips']#'regionidzip','regionidneighborhood',
cnts     = ['bathroomcnt', 'fullbathcnt', 'bedroomcnt','threequarterbathnbr',
            'numberofstories','garagecarcnt','roomcnt','fireplacecnt','calculatedbathnbr',
            'unitcnt','poolcnt']
surfaces = ['finishedsquarefeet12', 'calculatedfinishedsquarefeet',
            'finishedsquarefeet50','basementsqft','lotsizesquarefeet',
            'finishedsquarefeet13','yardbuildingsqft17','garagetotalsqft',
            'finishedfloor1squarefeet','yardbuildingsqft26','finishedsquarefeet15',
            'poolsizesum','finishedsquarefeet6']
taxes    = ['structuretaxvaluedollarcnt','taxvaluedollarcnt','taxdelinquencyflag','taxdelinquencyyear',
            'landtaxvaluedollarcnt','taxamount']

# Columns to be binarised.
dummies = ['taxdelinquencyyear'] + regional + ids
#descript = ['propertyzoningdesc']
other    = ['age','latitude','longitude']#'yearbuilt']

transactions_final_columns = temporal + ids + regional + cnts + surfaces + taxes + other

# Building time features
transactions['transactiondate'] = pd.to_datetime(transactions['transactiondate'])
transactions['day_of_week'] = transactions['transactiondate'].dt.dayofweek
transactions['month_of_year'] = transactions['transactiondate'].dt.month
transactions['quarter'] = transactions['transactiondate'].dt.quarter
transactions['is_weekend'] = (transactions['day_of_week'] < 5).astype(int)
transactions['year'] = transactions['transactiondate'].dt.year
transactions['age'] = transactions['year'] - transactions['yearbuilt']
# Other features
transactions['taxdelinquencyflag'] = transactions['taxdelinquencyflag'].replace('Y', 1)
transactions['code_fips']  = transactions['rawcensustractandblock'].apply(lambda x: str(x)[:4])
transactions['code_tract'] = transactions['rawcensustractandblock'].apply(lambda x: str(x)[4:11])
transactions['code_block'] = transactions['rawcensustractandblock'].apply(lambda x: str(x)[11:])
# Feature importance based on its correlation with the 'logerror'
# corrs = transactions.corr()['logerror'].sort_values(ascending=False)
transactions = transactions.fillna(0)
# Build train features
transactions_shuffled = transactions.sample(frac=1)

# Get sparse data
x_all = pd.get_dummies(transactions_shuffled[transactions_final_columns], columns=dummies)
#x_all = transactions_shuffled[transactions_final_columns]
y_all = transactions_shuffled['logerror'].values

new_sparse_columns = x_all.columns
# Splits up train and test based on the given ration
ratio = 0.1
x_train, x_test, y_train, y_test = train_test_split(x_all.values, y_all, test_size=ratio, random_state=69)

In [ ]:
# Show transaction distribution over time.
fig = plt.figure(figsize=(20,9))
counts = transactions.groupby(['month_of_year','year']).count()['parcelid'].reset_index()
plt.bar(counts['month_of_year'].values, counts['parcelid'].values)
plt.title('Transaction ditribution over 12 months.')

# Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

model_lr = LinearRegression()
# Fir the model to train data
model_lr.fit(x_train, y_train)
# Make a prediction for the test set
y_pred_lr_test = model_lr.predict(x_test)
y_pred_lr_train = model_lr.predict(x_train)
# Score the predictor on the test set
model_lr.score(x_test, y_test)
# Feature importance
feature_importance_lr = pd.DataFrame(model_lr.coef_, columns=['Weight'], index=new_sparse_columns).sort_values('Weight', ascending=False)
# Same R2 computation based but based on metrics library
print('R2 LR Test:', r2_score(y_test, y_pred_lr_test), 'Train:', r2_score(y_train, y_pred_lr_train))

predicted_mae_lr_test = mean_absolute_error(y_test, y_pred_lr_test)
predicted_mae_lr_train = mean_absolute_error(y_train, y_pred_lr_train)
print('MAE LR  Train:', predicted_mae_lr_train, 'Test:',predicted_mae_lr_test)

sample = 100 # Number of records to look at - makes the visualisation more meaningful.
# Plot test true vs predicted values
plot_data(y_test, y_pred_lr_test, sample, 'Test', linewidth=2)
# Plot train true vs predicted values
plot_data(y_train, y_pred_lr_train, sample, 'Train', linewidth=2)

# XGB

In [ ]:
import xgboost as xgb
from xgboost import XGBModel
from xgboost import plot_importance
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import RFE

def plot_best_features(model, data, num_features, figsize=(5,50)):
    """ 
    Plot best features. 
    
    Args:
        model (XGBRegressor) : The best XGBRegressor estimator from GridSearchCV or other 
                               model of type XGBRegressor.
        data     (DataFrame) : Data containing all features and column names.
        
    Returns:
        dict : The newly created dictionary, which maps 'data' features to their associated score.
        
    """
    
    new_scores = {}
    # Get the XGB Model's score and assign values and keys to the new dictionary
    scores = model.booster().get_score(importance_type='weight')
    for i in scores.keys():
        new_scores[data.columns[int(i[1:])]] = scores[i]
    # Build a dataframe with the top 'num_features'
    df_features = pd.DataFrame.from_records([new_scores], index=['Features']).T.sort_values('Features').tail(num_features)
    # Plot feature significance based on the models' score
    df_features.plot.barh(figsize=figsize)
    
    return new_scores


cv = 5
jobs = 10
params={
    'max_depth':        [3], # shuld be 0.5 to 1% of the examples
    'subsample':        [0.8, 1], #[0.4,0.5,0.6,0.7,0.8,0.9,1.0],
    'min_child_weight': [10],
    #'colsample_bytree': [0.5], #[0.5,0.6,0.7,0.8],
    'objective':        ['reg:linear'],
    'n_estimators':     [1000], #[1000,2000,3000]
    'reg_alpha':        [0], #[0.01, 0.02, 0.03, 0.04]
    'learning_rate':    [0.1]
}

# Build XGB model based on the given parameters.
# Default features:
# max_depth=3, learning_rate=0.1, n_estimators=100, silent=True, objective='reg:linear', 
# booster='gbtree', n_jobs=1, nthread=None, gamma=0, min_child_weight=1, max_delta_step=0, 
# subsample=1, colsample_bytree=1, colsample_bylevel=1, reg_alpha=0, reg_lambda=1, 
# scale_pos_weight=1, base_score=0.5, random_state=0, seed=None, missing=None
xgbr = xg.XGBRegressor()
xgb_gs =GridSearchCV(xgbr, params, n_jobs=jobs, 
                   cv=TimeSeriesSplit(n_splits=cv).get_n_splits([x_train,y_train]), 
                   #scoring='neg_mean_absolute_error',
                   verbose=1, refit=True)

xgb_gs.fit(x_train, y_train)
print('Best estimator:',xgb_gs.best_estimator_)
# Predict estimated logerror
y_pred_xgb_test = xgb_gs.predict(x_test)
y_pred_xgb_train = xgb_gs.predict(x_train)
# Evaluate the performance of XGB
print('XGB R2 Test:', xgb_gs.score(x_test, y_test), 'Train:', xgb_gs.score(x_train, y_train))

# Show results for LR on train and test data
print('MAE LR  Train:', predicted_mae_lr_train, 'Test:',predicted_mae_lr_test)

# Show results for XGB on train and test data
predicted_mae_xgb_test = mean_absolute_error(y_test, y_pred_xgb_test)
predicted_mae_xgb_train = mean_absolute_error(y_train, y_pred_xgb_train)
print('MAE XGB Train:', predicted_mae_xgb_train, 'Test:',predicted_mae_xgb_test)

#selector = RFE(xgb_gs.best_estimator_, 100, step=50)
#selector = selector.fit(x_train, y_train)

# plot feature importance
dict_features = plot_best_features(xgb_gs.best_estimator_, data=x_all, num_features=50, figsize=(5,13))
#xgb.to_graphviz(xgb_gs.best_estimator_, num_trees=50)

######## ONE-HOT vs NONE ######
# ONE-HOT:
# XGB R2 Test: -0.0135715900237 Train: 0.13302709163
# MAE LR  Train: 0.0687819933569 Test: 0.0675078152125
# MAE XGB Train: 0.0669039183129 Test: 0.0687189110259

# NONE:
# XGB R2 Test: 0.011877997183 Train: 0.141170034298
# MAE LR  Train: 0.0680096936624 Test: 0.0713713207428
# MAE XGB Train: 0.0662614095391 Test: 0.0722176694789

In [ ]:
submission_sample.head()